## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Mar. 2nd Mar. 9th


In [14]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [15]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [16]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [17]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [18]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,March-7-2025,Condo/Co-op,1408 Brickell Bay Dr #1216,Miami,FL,33131.0,285000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1408-Brickell-...,MARMLS,A11741365,N,Y,25.759105,-80.189261


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [19]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [20]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [21]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-03-03    33
2025-03-04    28
2025-03-07    22
2025-03-05    18
2025-03-06    16
Name: count, dtype: int64

In [22]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [23]:
df_top_ten = df2.head(10)

In [24]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
67,PAST SALE,2025-03-07,Condo/Co-op,17749 Collins Ave #502,Sunny Isles Beach,FL,33160.0,6800000.0,3.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11696879,N,Y,25.940065,-80.120264
14,PAST SALE,2025-03-04,Condo/Co-op,2821 S Bayshore Dr Unit 7A,Miami,FL,33133.0,6650000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,MARMLS,A11737670,N,Y,25.729608,-80.239162
117,PAST SALE,2025-03-07,Condo/Co-op,17141 Collins Ave #2801,Sunny Isles Beach,FL,33160.0,6500000.0,6.0,7.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11382107,N,Y,25.934939,-80.120918
114,PAST SALE,2025-03-07,Condo/Co-op,1135 103rd St #703,Bay Harbor Islands,FL,33154.0,5400000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11519325,N,Y,25.894019,-80.132182
12,PAST SALE,2025-03-04,Condo/Co-op,1135 103rd St #601,Bay Harbor Islands,FL,33154.0,5400000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11744804,N,Y,25.894019,-80.132182
108,PAST SALE,2025-03-04,Condo/Co-op,1445 16th St Ph -1,Miami Beach,FL,33139.0,5025000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1445-16t...,MARMLS,A11564126,N,Y,25.789262,-80.144721
115,PAST SALE,2025-03-04,Condo/Co-op,15701 Collins Ave #601,Sunny Isles Beach,FL,33160.0,4550000.0,4.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11503873,N,Y,25.921944,-80.121716
79,PAST SALE,2025-03-03,Condo/Co-op,6901 Collins Ave #802,Miami Beach,FL,33141.0,4400000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6901-Col...,MARMLS,A11670486,N,Y,25.854403,-80.120212
38,PAST SALE,2025-03-05,Condo/Co-op,16901 Collins Ave #3403,Sunny Isles Beach,FL,33160.0,4260000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11719530,N,Y,25.932764,-80.121036
72,PAST SALE,2025-03-05,Condo/Co-op,16901 Collins Ave #3303,Sunny Isles Beach,FL,33160.0,4000000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11673489,N,Y,25.932764,-80.121036


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [25]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [26]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [27]:
response_df = pd.DataFrame(response_list)

In [28]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [29]:
df_top_ten = merged_df

In [30]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-03-07,Condo/Co-op,17749 Collins Ave #502,Sunny Isles Beach,FL,33160.0,6800000.0,3.0,6.5,...,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ludmila Bogatov,Bogatov Realty,None,None,Yuriy Ustoyev,Encore Alliance Realty,None,None
1,PAST SALE,2025-03-04,Condo/Co-op,2821 S Bayshore Dr Unit 7A,Miami,FL,33133.0,6650000.0,4.0,5.5,...,-80.239162,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,Daniela Zeitel Helcer,Douglas Elliman,None,None,Ashley Cusack,BHHS EWM Realty,None,None
2,PAST SALE,2025-03-07,Condo/Co-op,17141 Collins Ave #2801,Sunny Isles Beach,FL,33160.0,6500000.0,6.0,7.5,...,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Mariana Niro,Serhant,None,None,Moshe Hankin,Wilrose Realty,None,None
3,PAST SALE,2025-03-07,Condo/Co-op,1135 103rd St #703,Bay Harbor Islands,FL,33154.0,5400000.0,3.0,4.5,...,-80.132182,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Christine Martinez de Castro,CMC Real Estate,None,None,Eloy Carmenate,The Corcoran Group,None,None
4,PAST SALE,2025-03-04,Condo/Co-op,1135 103rd St #601,Bay Harbor Islands,FL,33154.0,5400000.0,4.0,4.5,...,-80.132182,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Constanza Fernandez,Fortune Christie's International Real Estate,None,None,3009174 Romanello,"Compass Florida, LLC.",Claudia Villar,"Compass Florida, LLC."
5,PAST SALE,2025-03-04,Condo/Co-op,1445 16th St Ph -1,Miami Beach,FL,33139.0,5025000.0,3.0,3.5,...,-80.144721,https://www.redfin.com/FL/Miami-Beach/1445-16t...,Chad Carroll,"Compass Florida, LLC",None,None,Raymond Janssen,Gary Hennes Realtors,None,None
6,PAST SALE,2025-03-04,Condo/Co-op,15701 Collins Ave #601,Sunny Isles Beach,FL,33160.0,4550000.0,4.0,4.5,...,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Carlo Dipasquale,"Compass Florida, LLC.",Wendy Mendoza,Cayhill Dumott Realty LLC,Yelena Zborovsky,Azora Realty,None,None
7,PAST SALE,2025-03-03,Condo/Co-op,6901 Collins Ave #802,Miami Beach,FL,33141.0,4400000.0,3.0,3.5,...,-80.120212,https://www.redfin.com/FL/Miami-Beach/6901-Col...,Ron Laviv,The Corcoran Group,None,None,Gabriela Dajer,One Sotheby's International Realty,None,None
8,PAST SALE,2025-03-05,Condo/Co-op,16901 Collins Ave #3403,Sunny Isles Beach,FL,33160.0,4260000.0,3.0,4.5,...,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Ekaterina Brosda,"Brosda & Bentley Realtors, LLC",None,None,Kelly Salinas,RE/MAX Advance Realty II,None,None
9,PAST SALE,2025-03-05,Condo/Co-op,16901 Collins Ave #3303,Sunny Isles Beach,FL,33160.0,4000000.0,3.0,4.5,...,-80.121036,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Christiane Brooks,"Compass Florida, LLC.",None,None,Ember Duran,The Agency Naples,None,None


## Current Week's Values

In [31]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

117


In [32]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$1,032,863


In [34]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$653


In [35]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$120,844,976


In [40]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [53]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PDF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
124
-----------
Input Previous Week Condo Average Sales Price:
1014817
-----------
Input Previous Week Condo Average PSF:
635
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
125837300.0


In [54]:
last_week

,Condo Sales,Average Sale Price,Average PDF,Last Week Total Dollar Volume
0,124,1014817,635,125837300.0


In [55]:
last_week['Last Week Total Dollar Volume'].iloc[0]

125837300.0

In [57]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0          117             1032863          653                    120844976.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [58]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [59]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [60]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [61]:
pd.set_option('display.max_columns',None)

In [62]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-07,Condo/Co-op,17749 Collins Ave #502,Sunny Isles Beach,FL,33160.0,6800000.0,3.0,6.5,THE MANSIONS AT ACQUALINA,4609.0,NaN,2015.0,NaN,1475.0,20576.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11696879,N,Y,25.940065,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ludmila Bogatov,Bogatov Realty,None,None,Yuriy Ustoyev,Encore Alliance Realty,None,None,1,orange


In [63]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [64]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-07,Condo/Co-op,17749 Collins Ave #502,Sunny Isles Beach,FL,33160.0,6800000.0,3.0,6.5,THE MANSIONS AT ACQUALINA,4609.0,NaN,2015.0,NaN,1475.0,20576.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11696879,N,Y,25.940065,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ludmila Bogatov,Bogatov Realty,None,None,Yuriy Ustoyev,Encore Alliance Realty,None,None,1,orange
1,PAST SALE,2025-03-04,Condo/Co-op,2821 S Bayshore Dr Unit 7A,Miami,FL,33133.0,6650000.0,4.0,5.5,TWO PARK GROVE CONDO,3523.0,NaN,2018.0,NaN,1888.0,6592.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,MARMLS,A11737670,N,Y,25.729608,-80.239162,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,Daniela Zeitel Helcer,Douglas Elliman,None,None,Ashley Cusack,BHHS EWM Realty,None,None,2,blue
2,PAST SALE,2025-03-07,Condo/Co-op,17141 Collins Ave #2801,Sunny Isles Beach,FL,33160.0,6500000.0,6.0,7.5,MUSE CONDO,5359.0,NaN,2018.0,NaN,1213.0,15266.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11382107,N,Y,25.934939,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Mariana Niro,Serhant,None,None,Moshe Hankin,Wilrose Realty,None,None,3,blue
3,PAST SALE,2025-03-07,Condo/Co-op,1135 103rd St #703,Bay Harbor Islands,FL,33154.0,5400000.0,3.0,4.5,Onda Residences,3097.0,NaN,2024.0,NaN,1744.0,6041.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11519325,N,Y,25.894019,-80.132182,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Christine Martinez de Castro,CMC Real Estate,None,None,Eloy Carmenate,The Corcoran Group,None,None,4,blue
4,PAST SALE,2025-03-04,Condo/Co-op,1135 103rd St #601,Bay Harbor Islands,FL,33154.0,5400000.0,4.0,4.5,Onda Residences,3174.0,NaN,2024.0,NaN,1701.0,6636.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11744804,N,Y,25.894019,-80.132182,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Constanza Fernandez,Fortune Christie's International Real Estate,None,None,3009174 Romanello,"Compass Florida, LLC.",Claudia Villar,"Compass Florida, LLC.",5,blue
5,PAST SALE,2025-03-04,Condo/Co-op,1445 16th St Ph -1,Miami Beach,FL,33139.0,5025000.0,3.0,3.5,CAPRI SOUTH BEACH CONDO,2970.0,NaN,2008.0,NaN,1692.0,4635.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1445-16t...,MARMLS,A11564126,N,Y,25.789262,-80.144721,https://www.redfin.com/FL/Miami-Beach/1445-16t...,Chad Carroll,"Compass Florida, LLC",None,None,Raymond Janssen,Gary Hennes Realtors,None,None,6,blue
6,PAST SALE,2025-03-04,Condo/Co-op,15701 Collins Ave #601,Sunny Isles Beach,FL,33160.0,4550000.0,4.0,4.5,15701 COLLINS CONDO,3080.0,NaN,2020.0,NaN,1477.0,4579.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11503873,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Carlo Dipasquale,"Compass Florida, LLC.",Wendy Mendoza,Cayhill Dumott Realty LLC,Yelena Zborovsky,Azora Realty,None,None,7,blue
7,PAST SALE,2025-03-03,Condo/Co-op,6901 Collins Ave #802,Miami Beach,FL,33141.0,4400000.0,3.0,3.5,L'ATELIER CONDO,2133.0,NaN,2019.0,NaN,2063.0,7835.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6901-Col...,MARMLS,A11670486,N,Y,25.854403,-80.120212,https://www.redfin.com/FL/Miami-Beach/6901-Col...,Ron Laviv,The Corcoran Group,None,None,Gabriela Dajer,One Sotheby's International Realty,None,None,8,blue
8,PAST SALE,2025-03-05,Condo/Co-op,16901 Collins Ave #3403,Sunny Isles Beach,FL,33160.0,4260000.0,3.0,4.5,JADE SIGNATURE CONDO,2511.0,NaN,2018.0,NaN,1697.0,5022.0,Sold,NaN,NaN,https://

In [65]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [66]:
m.save('index.html')

## Data snagger

In [67]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [68]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-03-07,Condo/Co-op,17749 Collins Ave #502,Sunny Isles Beach,FL,33160.0,6800000.0,3.0,6.5,THE MANSIONS AT ACQUALINA,4609.0,NaN,2015.0,NaN,1475.0,20576.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11696879,N,Y,25.940065,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ludmila Bogatov,Bogatov Realty,None,None,Yuriy Ustoyev,Encore Alliance Realty,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [69]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_03102024


# CREATE TEMPLATE 

In [70]:
muni_set = set(df_top_ten['CITY'])

In [71]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [72]:
df_top_ten.reset_index(inplace=True,drop=True)

In [73]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [74]:
top_sale

'THE MANSIONS AT ACQUALINA at 17749 Collins Ave #502 in Sunny Isles Beach'

In [75]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [76]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [77]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-07,Condo/Co-op,17749 Collins Ave #502,Sunny Isles Beach,FL,33160.0,6800000.0,3.0,6.5,THE MANSIONS AT ACQUALINA,4609.0,NaN,2015.0,NaN,1475.0,20576.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11696879,N,Y,25.940065,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ludmila Bogatov,Bogatov Realty,None,None,Yuriy Ustoyev,Encore Alliance Realty,None,None,1,orange,THE MANSIONS AT ACQUALINA at 17749 Collins Ave...
1,PAST SALE,2025-03-04,Condo/Co-op,2821 S Bayshore Dr Unit 7A,Miami,FL,33133.0,6650000.0,4.0,5.5,TWO PARK GROVE CONDO,3523.0,NaN,2018.0,NaN,1888.0,6592.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,MARMLS,A11737670,N,Y,25.729608,-80.239162,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,Daniela Zeitel Helcer,Douglas Elliman,None,None,Ashley Cusack,BHHS EWM Realty,None,None,2,blue,TWO PARK GROVE CONDO at 2821 S Bayshore Dr Uni...
2,PAST SALE,2025-03-07,Condo/Co-op,17141 Collins Ave #2801,Sunny Isles Beach,FL,33160.0,6500000.0,6.0,7.5,MUSE CONDO,5359.0,NaN,2018.0,NaN,1213.0,15266.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11382107,N,Y,25.934939,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Mariana Niro,Serhant,None,None,Moshe Hankin,Wilrose Realty,None,None,3,blue,MUSE CONDO at 17141 Collins Ave #2801 in Sunny...
3,PAST SALE,2025-03-07,Condo/Co-op,1135 103rd St #703,Bay Harbor Islands,FL,33154.0,5400000.0,3.0,4.5,Onda Residences,3097.0,NaN,2024.0,NaN,1744.0,6041.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11519325,N,Y,25.894019,-80.132182,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Christine Martinez de Castro,CMC Real Estate,None,None,Eloy Carmenate,The Corcoran Group,None,None,4,blue,Onda Residences at 1135 103rd St #703 in Bay H...
4,PAST SALE,2025-03-04,Condo/Co-op,1135 103rd St #601,Bay Harbor Islands,FL,33154.0,5400000.0,4.0,4.5,Onda Residences,3174.0,NaN,2024.0,NaN,1701.0,6636.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11744804,N,Y,25.894019,-80.132182,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Constanza Fernandez,Fortune Christie's International Real Estate,None,None,3009174 Romanello,"Compass Florida, LLC.",Claudia Villar,"Compass Florida, LLC.",5,blue,Onda Residences at 1135 103rd St #601 in Bay H...
5,PAST SALE,2025-03-04,Condo/Co-op,1445 16th St Ph -1,Miami Beach,FL,33139.0,5025000.0,3.0,3.5,CAPRI SOUTH BEACH CONDO,2970.0,NaN,2008.0,NaN,1692.0,4635.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1445-16t...,MARMLS,A11564126,N,Y,25.789262,-80.144721,https://www.redfin.com/FL/Miami-Beach/1445-16t...,Chad Carroll,"Compass Florida, LLC",None,None,Raymond Janssen,Gary Hennes Realtors,None,None,6,blue,CAPRI SOUTH BEACH CONDO at 1445 16th St Ph -1 ...
6,PAST SALE,2025-03-04,Condo/Co-op,15701 Collins Ave #601,Sunny Isles Beach,FL,33160.0,4550000.0,4.0,4.5,15701 COLLINS CONDO,3080.0,NaN,2020.0,NaN,1477.0,4579.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11503873,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Carlo Dipasquale,"Compass Florida, LLC.",Wendy Mendoza,Cayhill Dumott Realty LLC,Yelena Zborovsky,Azora Realty,None,None,7,blue,15701 COLLINS CONDO at 15701 Collins Ave #601 ...
7,PAST SALE,2025-03-03,Condo/Co-op,6901 Collins Ave #802,Miami Beach,FL,33141.0,4400000.0,3.0,3.5,L'ATELIER CONDO,2133.0,NaN,2019.0,NaN,2063.0,7835.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6901-Col...,MARMLS,A11670486,N,Y,2

In [78]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: THE MANSIONS AT ACQUALINA closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $4,000,000 to $6,800,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Bay Harbor Islands, Sunny Isles Beach, Miami, Miami Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 117 condo sales totaling $120,844,976 million from Mar. 2nd to Mar. 9th. The previous week, brokers closed 124 condo sales totaling $125,837,300.

Last week’s units sold for an average of $1,032,863, higher than the $1,014,817 averag

In [79]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [80]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [81]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [82]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-07,Condo/Co-op,17749 Collins Ave #502,Sunny Isles Beach,FL,33160.0,6800000.0,3.0,6.5,THE MANSIONS AT ACQUALINA,4609.0,NaN,2015.0,NaN,1475.0,20576.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11696879,N,Y,25.940065,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ludmila Bogatov,Bogatov Realty,None,None,Yuriy Ustoyev,Encore Alliance Realty,None,None,1,orange,THE MANSIONS AT ACQUALINA at 17749 Collins Ave...
1,PAST SALE,2025-03-04,Condo/Co-op,2821 S Bayshore Dr Unit 7A,Miami,FL,33133.0,6650000.0,4.0,5.5,TWO PARK GROVE CONDO,3523.0,NaN,2018.0,NaN,1888.0,6592.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,MARMLS,A11737670,N,Y,25.729608,-80.239162,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,Daniela Zeitel Helcer,Douglas Elliman,None,None,Ashley Cusack,BHHS EWM Realty,None,None,2,blue,TWO PARK GROVE CONDO at 2821 S Bayshore Dr Uni...
2,PAST SALE,2025-03-07,Condo/Co-op,17141 Collins Ave #2801,Sunny Isles Beach,FL,33160.0,6500000.0,6.0,7.5,MUSE CONDO,5359.0,NaN,2018.0,NaN,1213.0,15266.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11382107,N,Y,25.934939,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Mariana Niro,Serhant,None,None,Moshe Hankin,Wilrose Realty,None,None,3,blue,MUSE CONDO at 17141 Collins Ave #2801 in Sunny...
3,PAST SALE,2025-03-07,Condo/Co-op,1135 103rd St #703,Bay Harbor Islands,FL,33154.0,5400000.0,3.0,4.5,Onda Residences,3097.0,NaN,2024.0,NaN,1744.0,6041.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11519325,N,Y,25.894019,-80.132182,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Christine Martinez de Castro,CMC Real Estate,None,None,Eloy Carmenate,The Corcoran Group,None,None,4,blue,Onda Residences at 1135 103rd St #703 in Bay H...
4,PAST SALE,2025-03-04,Condo/Co-op,1135 103rd St #601,Bay Harbor Islands,FL,33154.0,5400000.0,4.0,4.5,Onda Residences,3174.0,NaN,2024.0,NaN,1701.0,6636.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11744804,N,Y,25.894019,-80.132182,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Constanza Fernandez,Fortune Christie's International Real Estate,None,None,3009174 Romanello,"Compass Florida, LLC.",Claudia Villar,"Compass Florida, LLC.",5,blue,Onda Residences at 1135 103rd St #601 in Bay H...
5,PAST SALE,2025-03-04,Condo/Co-op,1445 16th St Ph -1,Miami Beach,FL,33139.0,5025000.0,3.0,3.5,CAPRI SOUTH BEACH CONDO,2970.0,NaN,2008.0,NaN,1692.0,4635.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1445-16t...,MARMLS,A11564126,N,Y,25.789262,-80.144721,https://www.redfin.com/FL/Miami-Beach/1445-16t...,Chad Carroll,"Compass Florida, LLC",None,None,Raymond Janssen,Gary Hennes Realtors,None,None,6,blue,CAPRI SOUTH BEACH CONDO at 1445 16th St Ph -1 ...
6,PAST SALE,2025-03-04,Condo/Co-op,15701 Collins Ave #601,Sunny Isles Beach,FL,33160.0,4550000.0,4.0,4.5,15701 COLLINS CONDO,3080.0,NaN,2020.0,NaN,1477.0,4579.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11503873,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Carlo Dipasquale,"Compass Florida, LLC.",Wendy Mendoza,Cayhill Dumott Realty LLC,Yelena Zborovsky,Azora Realty,None,None,7,blue,15701 COLLINS CONDO at 15701 Collins Ave #601 ...
7,PAST SALE,2025-03-03,Condo/Co-op,6901 Collins Ave #802,Miami Beach,FL,33141.0,4400000.0,3.0,3.5,L'ATELIER CONDO,2133.0,NaN,2019.0,NaN,2063.0,7835.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6901-Col...,MARMLS,A11670486,N,Y,2

In [84]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: THE MANSIONS AT ACQUALINA closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $4,000,000 to $6,800,000
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Bay Harbor Islands, Sunny Isles Beach, Miami, Miami Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 117 condo sales totaling $120,844,976 million from Mar. 2nd to Mar. 9th. The previous week, brokers closed 124 condo sales totaling $125,837,300.

Last week’s units sold for an average of $1,032,863, higher than the $1,014,817 averag

In [92]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Sunny-Isles-Beach/17749-Collins-Ave-33160/unit-502/home/110130725


In [93]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami/2821-S-Bayshore-Dr-33133/unit-7A/home/79134088


In [94]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Sunny-Isles-Beach/16901-Collins-Ave-33160/unit-3303/home/56743940


In [95]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Sunny-Isles-Beach/16901-Collins-Ave-33160/unit-3403/home/56743941


In [96]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/6901-Collins-Ave-33141/unit-802/home/168031496


In [97]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Sunny-Isles-Beach/17141-Collins-Ave-33160/unit-2801/home/145501732


In [98]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-07,Condo/Co-op,17749 Collins Ave #502,Sunny Isles Beach,FL,33160.0,6800000.0,3.0,6.5,THE MANSIONS AT ACQUALINA,4609.0,NaN,2015.0,NaN,1475.0,20576.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11696879,N,Y,25.940065,-80.120264,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Ludmila Bogatov,Bogatov Realty,None,None,Yuriy Ustoyev,Encore Alliance Realty,None,None,1,orange,THE MANSIONS AT ACQUALINA at 17749 Collins Ave...
1,PAST SALE,2025-03-04,Condo/Co-op,2821 S Bayshore Dr Unit 7A,Miami,FL,33133.0,6650000.0,4.0,5.5,TWO PARK GROVE CONDO,3523.0,NaN,2018.0,NaN,1888.0,6592.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,MARMLS,A11737670,N,Y,25.729608,-80.239162,https://www.redfin.com/FL/Miami/2821-S-Bayshor...,Daniela Zeitel Helcer,Douglas Elliman,None,None,Ashley Cusack,BHHS EWM Realty,None,None,2,blue,TWO PARK GROVE CONDO at 2821 S Bayshore Dr Uni...
2,PAST SALE,2025-03-07,Condo/Co-op,17141 Collins Ave #2801,Sunny Isles Beach,FL,33160.0,6500000.0,6.0,7.5,MUSE CONDO,5359.0,NaN,2018.0,NaN,1213.0,15266.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11382107,N,Y,25.934939,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Mariana Niro,Serhant,None,None,Moshe Hankin,Wilrose Realty,None,None,3,blue,MUSE CONDO at 17141 Collins Ave #2801 in Sunny...
3,PAST SALE,2025-03-07,Condo/Co-op,1135 103rd St #703,Bay Harbor Islands,FL,33154.0,5400000.0,3.0,4.5,Onda Residences,3097.0,NaN,2024.0,NaN,1744.0,6041.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11519325,N,Y,25.894019,-80.132182,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Christine Martinez de Castro,CMC Real Estate,None,None,Eloy Carmenate,The Corcoran Group,None,None,4,blue,Onda Residences at 1135 103rd St #703 in Bay H...
4,PAST SALE,2025-03-04,Condo/Co-op,1135 103rd St #601,Bay Harbor Islands,FL,33154.0,5400000.0,4.0,4.5,Onda Residences,3174.0,NaN,2024.0,NaN,1701.0,6636.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,MARMLS,A11744804,N,Y,25.894019,-80.132182,https://www.redfin.com/FL/Bay-Harbor-Islands/1...,Constanza Fernandez,Fortune Christie's International Real Estate,None,None,3009174 Romanello,"Compass Florida, LLC.",Claudia Villar,"Compass Florida, LLC.",5,blue,Onda Residences at 1135 103rd St #601 in Bay H...
5,PAST SALE,2025-03-04,Condo/Co-op,1445 16th St Ph -1,Miami Beach,FL,33139.0,5025000.0,3.0,3.5,CAPRI SOUTH BEACH CONDO,2970.0,NaN,2008.0,NaN,1692.0,4635.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1445-16t...,MARMLS,A11564126,N,Y,25.789262,-80.144721,https://www.redfin.com/FL/Miami-Beach/1445-16t...,Chad Carroll,"Compass Florida, LLC",None,None,Raymond Janssen,Gary Hennes Realtors,None,None,6,blue,CAPRI SOUTH BEACH CONDO at 1445 16th St Ph -1 ...
6,PAST SALE,2025-03-04,Condo/Co-op,15701 Collins Ave #601,Sunny Isles Beach,FL,33160.0,4550000.0,4.0,4.5,15701 COLLINS CONDO,3080.0,NaN,2020.0,NaN,1477.0,4579.0,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/15...,MARMLS,A11503873,N,Y,25.921944,-80.121716,https://www.redfin.com/FL/North-Miami-Beach/15...,Carlo Dipasquale,"Compass Florida, LLC.",Wendy Mendoza,Cayhill Dumott Realty LLC,Yelena Zborovsky,Azora Realty,None,None,7,blue,15701 COLLINS CONDO at 15701 Collins Ave #601 ...
7,PAST SALE,2025-03-03,Condo/Co-op,6901 Collins Ave #802,Miami Beach,FL,33141.0,4400000.0,3.0,3.5,L'ATELIER CONDO,2133.0,NaN,2019.0,NaN,2063.0,7835.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/6901-Col...,MARMLS,A11670486,N,Y,2

## Time on Market Calculator

In [101]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2025, 2, 4) ## List (Earlier) date
date2 = datetime(2025, 3, 4) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

28


## Clean CSV for Datawrapper Chart

In [102]:
chart_df = df_top_ten

In [103]:
chart_df = chart_df.fillna(" ")

In [104]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [105]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [106]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [107]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [108]:
chart_df['$/SQUARE FEET'].astype(float)

0    1475.0
1    1888.0
2    1213.0
3    1744.0
4    1701.0
5    1692.0
6    1477.0
7    2063.0
8    1697.0
9    1593.0
Name: $/SQUARE FEET, dtype: float64

In [109]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [110]:
csv_date_string = today.strftime("%m_%d_%Y")

In [111]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [112]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-03-07,17749 Collins Ave #502 in Sunny Isles Beach,"$6,800,000",3,6.5,"4,609",2015,"$1,475",MARMLS,A11696879,Ludmila Bogatov Bogatov Realty,Yuriy Ustoyev Encore Alliance Realty
1,2025-03-04,2821 S Bayshore Dr Unit 7A in Miami,"$6,650,000",4,5.5,"3,523",2018,"$1,888",MARMLS,A11737670,Daniela Zeitel Helcer Douglas Elliman,Ashley Cusack BHHS EWM Realty
2,2025-03-07,17141 Collins Ave #2801 in Sunny Isles Beach,"$6,500,000",6,7.5,"5,359",2018,"$1,213",MARMLS,A11382107,Mariana Niro Serhant,Moshe Hankin Wilrose Realty
3,2025-03-07,1135 103rd St #703 in Bay Harbor Islands,"$5,400,000",3,4.5,"3,097",2024,"$1,744",MARMLS,A11519325,Christine Martinez de Castro CMC Real Estate,Eloy Carmenate The Corcoran Group
4,2025-03-04,1135 103rd St #601 in Bay Harbor Islands,"$5,400,000",4,4.5,"3,174",2024,"$1,701",MARMLS,A11744804,Constanza Fernandez Fortune Christie's Interna...,"3009174 Romanello Compass Florida, LLC. Claudi..."
5,2025-03-04,1445 16th St Ph -1 in Miami Beach,"$5,025,000",3,3.5,"2,970",2008,"$1,692",MARMLS,A11564126,"Chad Carroll Compass Florida, LLC",Raymond Janssen Gary Hennes Realtors
6,2025-03-04,15701 Collins Ave #601 in Sunny Isles Beach,"$4,550,000",4,4.5,"3,080",2020,"$1,477",MARMLS,A11503873,"Carlo Dipasquale Compass Florida, LLC. Wendy M...",Yelena Zborovsky Azora Realty
7,2025-03-03,6901 Collins Ave #802 in Miami Beach,"$4,400,000",3,3.5,"2,133",2019,"$2,063",MARMLS,A11670486,Ron Laviv The Corcoran Group,Gabriela Dajer One Sotheby's International Rea...
8,2025-03-05,16901 Collins Ave #3403 in Sunny Isles Beach,"$4,260,000",3,4.5,"2,511",2018,"$1,697",MARMLS,A11719530,"Ekaterina Brosda Brosda & Bentley Realtors, LL...",Kelly Salinas RE/MAX Advance Realty II
9,2025-03-05,16901 Collins Ave #3303 in Sunny Isles Beach,"$4,000,000",3,4.5,"2,511",2018,"$1,593",MARMLS,A11673489,"Christiane Brooks Compass Florida, LLC.",Ember Duran The Agency Naples
